# Pneumonia detection from X-ray

## downloading tensorflow for the project
##### run the following command in a code sell to download tensorflow



```terminal
!pip install tensorflow
```



### transfer learning is used to make this project 
####  <u>Transfer learning</u> :   Transfer Learning is basically when you train a model on a data and then used that trained model on a diffrent but similer data so that you don't have to train the entire model , which may take a lot of time and GPU power and for a good preformance will also required a large data set, which we know may not be peresent and making our own data set is tedious and expencive task

##### we will be using VGG16 and VGG19

#### you can donload the data set form [here](https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia)

In [3]:
import tensorflow as tf

# Check if GPU is available and set the device
if tf.config.list_physical_devices('GPU'):
    print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
    device_name = tf.test.gpu_device_name()
    print(f"GPU device name: {device_name}")
else:
    print("No GPU found, using CPU.")

# Example TensorFlow operations using GPU
with tf.device('/GPU:0'):
    a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
    b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
    c = tf.matmul(a, b)

print(c)

# Monitor GPU utilization
!nvidia-smi


Num GPUs Available:  2
GPU device name: /device:GPU:0
tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Wed Jul  3 15:11:41 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0              27W /  70W |  14065MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [6]:
# importing all the necessary libraries
import tensorflow as tf
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential 
import numpy as np
import glob as glob
import matplotlib.pyplot as plt

In [24]:
# re-sizing all the images ,as we are delaing with rgb images so it will have 3 dimentions
IMAGE_SIZE = [244, 244]

train_path = '/kaggle/input/chest-xray-pneumonia/chest_xray/train'
valid_path = '/kaggle/input/chest-xray-pneumonia/chest_xray/val'

In [9]:
# using vgg16 model , and we will be using imagenet weights, as they were the one who win the competion and so ther weights 
# are of the trained modle so we won't have to train the whole model , this is transfer learnig 
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights= 'imagenet', include_top=False)

# [3] is for the dimention of the rgb imgae

# include_top = False means we don't want the exact model as we are going to use a diffrent data set so , we will train 
# the the last layer and the output layer for out dataset so that the model predicts for this particular data set
# in the vgg16 starting layers of the cnn identify the basic pattrens that are persent in all the images like lines, edges etc, 
# so we don't need to change the staring layers for that we have keept them as same and removed the top part that we will be 
# training for out data set

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


In [10]:
# we are loading a pretrained model so we don't want to change the weights that we have laoded in the avobe code cell 
for layer in vgg.layers:
        layer.trainable = False    # so that we don't train the layers

In [25]:
from glob import glob
folders = glob("/kaggle/input/chest-xray-pneumonia/chest_xray/train/*")

# this will tell us which folders are present in Data_set/train , * repersents all , 

In [26]:
folders

['/kaggle/input/chest-xray-pneumonia/chest_xray/train/PNEUMONIA',
 '/kaggle/input/chest-xray-pneumonia/chest_xray/train/NORMAL']

In [27]:
x = Flatten()(vgg.output)
# here flatten is bsically reducing the dimentionality of the layers , making it in 1D
# vgg.output is the output layer and we are flattening the output layre

In [28]:
prediction = Dense(len(folders), activation= "softmax")(x)
# x is our output layer and we are usign Dense function to five it no. of neurons that there are in the train file
# for this case there are only two , pneumonia and not pneumonia 
# we are using activatin function softmax

model = Model(inputs = vgg.input, outputs = prediction)
# we are giving the model the inputs that are pretrained form vgg and outputs according to out data set

In [29]:
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 244, 244, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 244, 244, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 244, 244, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 122, 122, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 122, 122, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 122, 122, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 61, 61, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 61, 61, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 61, 61, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 61, 61, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 30, 30, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 30, 30, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 30, 30, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 30, 30, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 15, 15, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 15, 15, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 15, 15, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 15, 15, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │        50,178 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,764,866 (56.32 MB)

 Trainable params: 50,178 (196.01 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [30]:
# now we will compile the model , i.e. giving it all the things it will need to train well and make good predictions 
from tensorflow.keras.optimizers import Adam
model.compile(loss = 'categorical_crossentropy', 
             optimizer = Adam(0.0001),
             metrics = ["accuracy"])

In [31]:
# using image generator to import the images form the data set


train_datagen = ImageDataGenerator(rescale=1./255,
                                  shear_range=0.2,
                                  zoom_range=0.2, 
                                  horizontal_flip=True)

# here we are configuring the ImageDataGenerator with the parameters like rescale and all so that our model performs well
# it is adviced that if all the data is in same range then its better for the model 
### the basic reason we are doing this is because we want to increase out tring data somehow so that our model doesn't overfit
### this data augmentation is helping us with generarating more data for the training 

# rescale = 1./255 is rescaling the image's pixel value that ranges form 0 to 255 by deviding by 255 we can make it in range of 
# [0,1], this is a convention for better training 

# shear_range=0.2 shearing means keeping one part of the image fixed and move the other part in one direction keeping the 
# image stationary, this can introduce diversity into the training data

# horizontal_flip is used to mirror image this helps in making the model invarient to horizontal flips


test_datagen = ImageDataGenerator(rescale = 1./255)

In [32]:
# training data
# make sure to provide the same target size as initialized before 
training_set = train_datagen.flow_from_directory("/kaggle/input/chest-xray-pneumonia/chest_xray/train",
                                                target_size=(244,244),
                                                batch_size=32,
                                                class_mode= "categorical")

# at once 32 images will be given for training and we have to categorize the images 
# target size is (244, 244) this is done to ensure that the size of all the images is same when being fed to the model


Found 5216 images belonging to 2 classes.


In [35]:
# doing the same for test data
test_set = test_datagen.flow_from_directory("/kaggle/input/chest-xray-pneumonia/chest_xray/test",
                                                target_size=(244,244),
                                                batch_size=32,
                                                class_mode= "categorical")

Found 624 images belonging to 2 classes.


In [38]:
print(len(test_set))
print(next(iter(test_set))[0].shape, next(iter(test_set))[1].shape)

20
(32, 244, 244, 3) (32, 2)


In [43]:
# now we train our model 
xray = model.fit(
    training_set,
    validation_data=test_set,
    epochs=25,
    steps_per_epoch=len(training_set),
    validation_steps=(len(test_set)-1)
)

Epoch 1/25
163/163 ━━━━━━━━━━━━━━━━━━━━ 124s 731ms/step - accuracy: 0.9633 - loss: 0.1056 - val_accuracy: 0.9260 - val_loss: 0.2123
Epoch 2/25
163/163 ━━━━━━━━━━━━━━━━━━━━ 0s 534us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.8750 - val_loss: 0.3062
Epoch 3/25
163/163 ━━━━━━━━━━━━━━━━━━━━ 128s 756ms/step - accuracy: 0.9653 - loss: 0.0998 - val_accuracy: 0.9178 - val_loss: 0.2309
Epoch 4/25
163/163 ━━━━━━━━━━━━━━━━━━━━ 0s 774us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0874
Epoch 5/25
163/163 ━━━━━━━━━━━━━━━━━━━━ 119s 695ms/step - accuracy: 0.9670 - loss: 0.0889 - val_accuracy: 0.9227 - val_loss: 0.2224
Epoch 6/25
163/163 ━━━━━━━━━━━━━━━━━━━━ 0s 909us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.8750 - val_loss: 0.3604
Epoch 7/25
163/163 ━━━━━━━━━━━━━━━━━━━━ 124s 725ms/step - accuracy: 0.9655 - loss: 0.0948 - val_accuracy: 0.9030 - val_loss: 0.2581
Epoch 8/25
163/163 ━━━━━━━━━━━━━━━━━━━━ 0s 572us/step - ac

In [46]:
# from keras.models import load_model
model.save("/kaggle/working/VGG16_pneumonia_epoch_25.h5")

In [2]:
from keras.models import load_model
loaded_model = load_model("VGG16_pneumonia.h5")

In [47]:
# image has to be converted into numbers --> array to before to be processed

img = image.load_img( "/kaggle/input/chest-xray-pneumonia/chest_xray/val/NORMAL/NORMAL2-IM-1436-0001.jpeg", target_size = (244,244))
x = image.img_to_array(img)
x = np.expand_dims(img, axis = 0)
img_data = preprocess_input(x)
ans = model.predict(img_data)

1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step


In [51]:
predicted_class = np.argmax(ans)
predicted_class

0

In [52]:
if predicted_class == 0:
    print("you dont have Pneumonia")
else:
    print("Sorry, you have Pneumonia")

you dont have Pneumonia
